flexcan_driver.c 함수 정리
```cpp
/*******************************************************************************
 * Private Functions
 ******************************************************************************/
static status_t FLEXCAN_StartSendData(
                    uint8_t instance,
                    uint8_t mb_idx,
                    const flexcan_data_info_t *tx_info,
                    uint32_t msg_id,
                    const uint8_t *mb_data,
                    bool isBlocking
                    );
static status_t FLEXCAN_StartRxMessageBufferData(
                    uint8_t instance,
                    uint8_t mb_idx,
                    flexcan_msgbuff_t *data,
                    bool isBlocking
                    );
static status_t FLEXCAN_StartRxMessageFifoData(
                    uint8_t instance,
                    flexcan_msgbuff_t *data,
                    bool isBlocking
                    );
static void FLEXCAN_CompleteTransfer(uint8_t instance, uint32_t mb_idx);
static void FLEXCAN_CompleteRxMessageFifoData(uint8_t instance);
#if FEATURE_CAN_HAS_DMA_ENABLE
static void FLEXCAN_CompleteRxFifoDataDMA(void *parameter,
                                          edma_chn_status_t status);
#endif
static void FLEXCAN_BitrateToTimeSeg(uint32_t bitrate,
                                     uint32_t clkFreq,
                                     flexcan_time_segment_t *timeSeg);
static inline void FLEXCAN_IRQHandlerRxFIFO(uint8_t instance, uint32_t mb_idx);
static void FLEXCAN_IRQHandlerRxMB(uint8_t instance, uint32_t mb_idx);
static inline void FLEXCAN_EnableIRQs(uint8_t instance);
#ifdef ERRATA_E10368
#if FEATURE_CAN_HAS_FD
static inline void FLEXCAN_Errata10368(uint8_t instance,
                                       const flexcan_user_config_t *data);
#endif /* FEATURE_CAN_HAS_FD */
#endif /* ERRATA_E10368 */
/*******************************************************************************
 * Code
 ******************************************************************************/

```
위 코드는 내부 Function으로 건들지 않는다.

```cpp
void  FLEXCAN_DRV_GetBitrate(uint8_t instance, flexcan_time_segment_t *bitrate)
void  FLEXCAN_DRV_GetBitrateFD(uint8_t instance, flexcan_time_segment_t *bitrate)
```
여기서 `uint8_t instance`는 CAN 디바이스 번호를 의미하며, MPC5748G는 총 8개의 CAN 디바이스가 있다.

위 두 함수는 NXP 보드에서 설정하는 CAN bitrate 변수를 얻어오는 것으로, 값은 포인터 구조체 `flexcan_time_segment_t`에 저장된다. 일반 CAN은 1번 함수만 기동하고 CAN FD는 1번 함수는 Arbitration phase, 2번 함수는 Data phase 의 bitrate를 추출한다

```cpp
typedef struct {
    uint32_t propSeg;         /*!< Propagation segment*/
    uint32_t phaseSeg1;       /*!< Phase segment 1*/
    uint32_t phaseSeg2;       /*!< Phase segment 2*/
    uint32_t preDivider;      /*!< Clock prescaler division factor*/
    uint32_t rJumpwidth;      /*!< Resync jump width*/
} flexcan_time_segment_t;
```
위는 추출한 비트레이트 세그먼트 구조체 구조이다.

```cpp
void  FLEXCAN_DRV_SetRxMaskType(uint8_t instance, flexcan_rx_mask_type_t type)
```
위 함수는 CAN에서 필터를 걸 때, 해당 필터를 `FLEXCAN_RX_MASK_GLOBAL` 형식으로 걸지, 
`FLEXCAN_RX_MASK_INDIVIDUAL`로 걸 지를 결정해주는 함수이다.
여기서 MPC5748G의 FlexCAN 구조를 잠깐 언급해야 하는데, 8개의 CAN 디바이스가 장착된 MPC5748G의 FlexCAN 모듈은 최대 96개의 Message Buffers(MB)로 구성할 수 있으며, 각 CAN 버스간 Message Buffers는 공유되지 않는다.   
이, Message Buffers의 경우 예제 코드에서는 CAN FD는 7로 설정했는데, 한 디바이스가 96개의 메세지 버퍼를 운용할 수 있는지, 아니면 모든 디바이스의 운용 메세지 버퍼 총 합이 96인지는 확인 해봐야 한다.(데이터 시트 상으로는 각 디바이스별로 96개씩인듯 하다.)   
그리고 MPC5748G는 RxFIFO를 통해 수신받은 message를 필터링 할 수 있으며, 이 필터 테이블은 최소 8테이블, 최대 128테이블까지 설정이 가능하다.(기본8개)   
RxFIFO 필터 개수별로 CAN ID를 필터링 하는것이 가능하니, 효율적인 코딩을 위해 위 함수는 `FLEXCAN_RX_MASK_INDIVIDUAL`로 설정한다.
글로벌 마스크쪽 함수는 3~4개 존재하는데 FIFO부터 막는것도 있고 MB에서 막는것도 있고 뭐 좀 복잡하니까 아래의 함수 하나로 끝낸다.   
추가지식 : FIFO는 TxFIFO와 RxFIFO 두가지 종류가 있고, 각각 데이터 송수신에 사용되는 레지스터 이다. 그리고 TxFIFO랑 RxFIFO는 같은 주소를 사용한다 -> 이게 버퍼가 Queue 형태로 붙어서 체인처럼 돌아서 Tx Rx하는 것으로 이해된다.   
이게 FIFO랑 Message buffer, Mailboxes 3가지가 같이 붙어서 테이블 차트가 구성되고, 이 차트에 `Individual` 필터 요소를 결정해 주는 듯 하다.  
(솔직히 뭔지 잘 모르겟다)
아무튼 FIFO가 8개로 설정하면 메세지 버퍼는 0번부터 7번까지 메세지 버퍼에 붙고 나머지 95번까지는 mailbox로 할당되는것 같다.   
그래서 예제에서 안정성 때문에 메세지 버퍼를 최대개수 -1 하는듯 하다???   
아무튼 예제는 MB = 7, FIFO = 8이니 Individual Elements는 0~7 -> 8개이다.
-> 혹시나 어렵다면 `FLEXCAN_GetMaxMsgBuffNum` 로 최대값을 미리 받아두자...

```cpp
status_t FLEXCAN_DRV_SetRxIndividualMask(
    uint8_t instance, //디바이스 번호
    flexcan_msgbuff_id_type_t id_type,  //ID포맷 -> 11bit STD, 29bit EXT
    uint8_t mb_idx, //표 -> Individual Elements 개수 확인 후 max값 추출
    uint32_t mask)

typedef enum {
    FLEXCAN_MSG_ID_STD,         /*!< Standard ID*/
    FLEXCAN_MSG_ID_EXT          /*!< Extended ID*/
} flexcan_msgbuff_id_type_t;
```
아무튼 마스크 거는 필터 매개변수는 저렇고... 여기서 중요한게 mask인데 이 마스크의 경우   
`0xC0000000|0x마스크` 이렇게 입력하는것으로 끝내자
-> 어떻게 동작하는지 함수 타고 들어갔는데 비트연산하다가 GG침

그리고 마스크 함수랑 같이 동작해야 하는 필터 함수는
```cpp
void FLEXCAN_DRV_ConfigRxFifo(
    uint8_t instance,   //CAN 디바이스 번호
    flexcan_rx_fifo_id_element_format_t id_format,  //필터 입력 포멧 -> 아래 A, B, C, D 있는데 그냥 A씁시다.
    const flexcan_id_table_t *id_filter_table)
    //필터 테이블, 마스크 필터는 함수를 8개 불러와야 하지만, 필터 함수는 필터 구조체 만들어서 함수 한번만 부르면 된다
    //뭔가 비합리적이다...


typedef enum {
    FLEXCAN_RX_FIFO_ID_FORMAT_A, /*!< One full ID (standard and extended) per ID Filter Table element.*/
    FLEXCAN_RX_FIFO_ID_FORMAT_B, /*!< Two full standard IDs or two partial 14-bit (standard and
                                      extended) IDs per ID Filter Table element.*/
    FLEXCAN_RX_FIFO_ID_FORMAT_C, /*!< Four partial 8-bit Standard IDs per ID Filter Table element.*/
    FLEXCAN_RX_FIFO_ID_FORMAT_D  /*!< All frames rejected.*/
} flexcan_rx_fifo_id_element_format_t;

typedef struct {
    bool isRemoteFrame;      /*!< Remote frame*/        //이거는 리모트 프레임 안쓰니까 false
    bool isExtendedFrame;    /*!< Extended frame*/      //STD로 할거니까 false
    uint32_t id;             /*!< Rx FIFO ID filter element*/   //여기가 필터 ID 넣는 부분
} flexcan_id_table_t;

```

아무튼.. `FLEXCAN_DRV_ConfigRxFifo` 로 필터 포멧이랑 필터 ID(구조체) 넣어주고,   
`FLEXCAN_DRV_SetRxIndividualMask`이 함수 8번 불러서 마스크 ID 넣어주면   
원하는 CAN ID 필터링이 수행된다.   
아래는 예제이고 여기까지 하자...

```cpp
/* ID Filter table */
flexcan_id_table_t filterTable[8]={};
uint16_t id_counter;

for(id_counter=0;id_counter<8;id_counter++)
{
    filterTable[id_counter].isRemoteFrame = false;
    filterTable[id_counter].isExtendedFrame = false;
    filterTable[id_counter].id = 0x111 ;
}
/* Configure RX FIFO ID filter table elements based on filter table defined above*/
FLEXCAN_DRV_ConfigRxFifo(INST_CANCOM1, FLEXCAN_RX_FIFO_ID_FORMAT_A, filterTable);
/* set individual masking type */
FLEXCAN_DRV_SetRxMaskType(INST_CANCOM1, FLEXCAN_RX_MASK_INDIVIDUAL);
for(id_counter=0;id_counter<8;id_counter++)
FLEXCAN_DRV_SetRxIndividualMask(INST_CANCOM1, FLEXCAN_MSG_ID_STD, id_counter, 0xC0000000|0x7FF);
```

```cpp
status_t FLEXCAN_DRV_Init(
   uint8_t instance,
   flexcan_state_t *state,
   const flexcan_user_config_t *data)
```
FlexCAN 디바이스를 초기화 해주는 함수 -> pinMUX로 코드 젠하면 알아서 해당 함수가 main.c 구문에서 호출된다.   
매개변수는 딱히 알 필요없다. 코드 젠(혹은 예제) 된대로 쓰면 된다.


```cpp
status_t FLEXCAN_DRV_ConfigTxMb(
    uint8_t instance,
    uint8_t mb_idx,
    const flexcan_data_info_t *tx_info,
    uint32_t msg_id)

status_t FLEXCAN_DRV_ConfigRxMb(
    uint8_t instance,
    uint8_t mb_idx,
    const flexcan_data_info_t *rx_info,
    uint32_t msg_id)
```

다음으로 RxFIFO 활성화(FLEXCAN_DRV_Init 함수가 그 기능 수행) 된 뒤, 메세지 버퍼를 활성화 하는 코드   
이게 메세지 버퍼랑 해당 버퍼에 들어갈 CAN 메세지 ID도 같이 매개변수로 받고 있어서 TX하기 전에 해당   
CAN 메세지 ID에 맞춰서 버퍼를 구성해줘야 한다..   
뭐가 좀 복잡하다...
이거는 configRxMb도 동일하게 동작한다.   
아무튼 Tx가 됫던 Rx가 됬던 해당 CAN ID는 config를 먼저 수행해야 한다는 점...

추가로 `const flexcan_data_info_t *rx_info` 이 부분은 Rx, TX별로 따로 설정하지 말고   
CAN 디바이스별로 설정하는게 올바라 보인다.  
Rx일때는 CAN FD였다가 Tx일때는 CAN 2.0A 인 경우는 발생하는게 이상하니 말이다...

```cpp
status_t FLEXCAN_DRV_Send(
    uint8_t instance,
    uint8_t mb_idx,
    const flexcan_data_info_t *tx_info,
    uint32_t msg_id,
    const uint8_t *mb_data)
```

CAN 데이터 보내는 함수 매개변수는   
`CAN 디바이스번호`, `메세지 버퍼 번호`, `CAN프레임 설정정보`, `CAN 메세지ID`, `CAN 메세지` 이다.   
지금 구조체 함수의 상태값 안적고 있는데 얘의 경우 return값은 정상이면 `STATUS_SUCCESS`이다.

```cpp
status_t FLEXCAN_DRV_Receive(
    uint8_t instance,
    uint8_t mb_idx,
    flexcan_msgbuff_t *data)


typedef struct {
    uint32_t cs;                        /*!< Code and Status*/
    uint32_t msgId;                     /*!< Message Buffer ID*/
    uint8_t data[64];                   /*!< Data bytes of the FlexCAN message*/
    uint8_t dataLen;                    /*!< Length of data in bytes */
} flexcan_msgbuff_t;
```
CAN 수신(RX) 함수도 TX함수와 동일한 구조는 아니고, 매개변수 중 `flexcan_msgbuff_t *data`이 중요하다
`flexcan_msgbuff_t *data`는 구조체 변수이고   
첫번째 cs 값은
```cpp
    FLEXCAN_MB_IDLE,     0 /*!< The MB is not used by any transfer. */
    FLEXCAN_MB_RX_BUSY,  1 /*!< The MB is used for a reception. */
    FLEXCAN_MB_TX_BUSY,  2 /*!< The MB is used for a transmission. */
```
이거 인거 같다.   
나머지는 `CAN 메세지 ID`, `CAN 메세지`, `can 메세지 길이` 이다.

```cpp
status_t FLEXCAN_DRV_Deinit(uint8_t instance)
```
CAN 채널 종료하는 함수

```cpp
status_t FLEXCAN_DRV_GetTransferStatus(uint8_t instance, uint8_t mb_idx)
```
CAN 메세지 버퍼의 상태를 출력하는 함수   
이걸 왜 쓰냐면 메세지 버퍼에 값이 있을때 해당 값을 수신 한 후   
다음 수신까지 기다려야 할 때   
```cpp
while((FLEXCAN_DRV_GetTransferStatus(INST_CANCOM1, RX_MAILBOX) == STATUS_BUSY)
```
이렇게 대기하는 코드 만드려고...

When the Rx FIFO is enabled, MBs 0–5 are used by the Rx FIFO engine.
An additional memory area occupied by MBs 6–37 depending on the Number Of Rx FIFO Filters setting, contains the ID filter table (configurable from 8 to 128 table elements) that specifies filtering criteria for accepting frames into the FIFO.
![img_rxFIFO](./images/2023-01-25%20174413.png)